In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

In [ ]:

url_path: str = "https://raw.githubusercontent.com/CoderContenidos/Data.Engineering/main/Semana%208/Datos_Microdesafio_Semana8_DE.csv"


## Informacion no Anonima

In [ ]:
data:pd.DataFrame = pd.read_csv(url_path, sep=";")
data

## Columnas a Anonimizar

In [ ]:
cols_anonimizar: list[str] = ["Comisionado","Fecha","Telefono"]

### Forma recomendada por Coderhouse
LINK -> https://pypi.org/project/anonymizedf/

In [ ]:
from anonymizedf.anonymizedf import anonymize

In [ ]:
an = anonymize(data)

In [ ]:
fake_df = (
    an
    .fake_names("Comisionado", chaining=True)
    .fake_dates("Fecha", chaining=True)
    .fake_whole_numbers("Telefono", chaining=True)
    .show_data_frame()
)

data = fake_df
data

### Forma que lo hubiera realizado teniendo en cuenta encryptaciones

In [ ]:
import hashlib

In [ ]:
def anonymize(value):
    if pd.isnull(value):
        return value
    if isinstance(value, (int, float)):
        value = str(value)
    return hashlib.sha256(value.encode()).hexdigest()[:12]

for col in cols_anonimizar:
    data[f"{col}_hash"] = data[col].apply(anonymize)

data

In [ ]:
data_final_hashed: pd.DataFrame = data.drop(
    [   'Comisionado',
        'Fake_Comisionado',
        'Fake_Fecha', 
        'Fake_Telefono'], axis=1
) 

data_final_anonn: pd.DataFrame = data.drop(
    [   'Comisionado',
        'Comisionado_hash',
        'Fecha_hash', 
        'Telefono_hash'], axis=1
)

In [ ]:
data_final_hashed = data_final_hashed.rename(columns={"Pais ":"Pais"}) 

In [ ]:
data_final_anonn = data_final_anonn.rename(columns={"Pais ":"Pais"})
data_final_anonn.columns

### Cualquiera de esas elecciones no influye en la ingesta de los datos a la base de datos


In [ ]:
import psycopg2 as pg
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

load_dotenv()

In [ ]:
# Genera un objeto connector
conn = pg.connect(
        user = os.getenv('REDSHIFT_USERNAME')
    ,   password = os.getenv('REDSHIFT_PASSWORD')
    ,   host = os.getenv('REDSHIFT_HOST')
    ,   port = os.getenv('REDSHIFT_PORT', '5439')
    ,   database = os.getenv('REDSHIFT_DBNAME')
)

schema = "andru_ocatorres_coderhouse"


TABLE_QUERY = """

CREATE TABLE IF NOT EXISTS 
    POLITICAS_2050 (
        Pais VARCHAR(100), 
        Reduccion_CO2 VARCHAR(100), 
        Incrmento_P VARCHAR(100), 
        Inversion_arboles VARCHAR(100), 
        Fecha VARCHAR(100),
        Telefono VARCHAR(100), 
        Fake_Comisionado VARCHAR(100), 
        Fake_Fecha VARCHAR(100),
        Fake_Telefono VARCHAR(100)
    );
""" 

conn.autocommit = True

try:
    with conn.cursor() as cursor:
        cursor.execute(TABLE_QUERY)

    conn.commit()
    print("Schema createad plus table") 
except Exception as e:
    conn.rollback()
    print(f"not able to create \n{e}")
finally:
    conn.close()

In [ ]:
# crea un objeto url para conectar con la warehouse
username = os.getenv('REDSHIFT_USERNAME')
password = os.getenv('REDSHIFT_PASSWORD')
host = os.getenv('REDSHIFT_HOST')
port = os.getenv('REDSHIFT_PORT', '5439')
dbname = os.getenv('REDSHIFT_DBNAME')

# Construct the connection URL
connection_url = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}"
db_engine = create_engine(connection_url)


In [ ]:
data_final_anonn.to_sql('POLITICAS_2050', db_engine, if_exists='replace', index=False)